## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
enlace

'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210414.ods'

In [5]:
try:
    datos_row = pd.read_excel(enlace,sheet_name='Comunicación')
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210414.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [6]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas con al menos 1 dosis,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,1468440,219200,564800,2252440,1970183,0.874688,1419209,550974,2021-04-13
1,Aragón,291645,45800,88700,426145,331419,0.777714,228547,102872,2021-04-13
2,Asturias,280535,42200,68000,390735,317019,0.811340,216006,101013,2021-04-13
3,Baleares,158310,24300,81400,264010,233042,0.882701,175460,57582,2021-04-13
4,Canarias,319380,47800,145200,512380,418843,0.817446,302770,116073,2021-04-13
5,Cantabria,132945,19900,40500,193345,147960,0.765264,101801,46159,2021-04-13
6,Castilla y Leon,647895,100800,166900,915595,730342,0.797669,492310,238032,2021-04-13
7,Castilla La Mancha,412875,63800,142400,619075,499581,0.806980,344341,155240,2021-04-13
8,Cataluña,1413980,221100,519100,2154180,1843205,0.855641,1381065,462140,2021-04-13
9,C. Valenciana,836740,131600,352300,1320640,1125277,0.852069,835363,289914,2021-04-13


..hacemos una copia para hacerle las transformaciones necesarias:

In [7]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [8]:
datos_hoy.columns

Index(['Unnamed: 0', 'Dosis entregadas Pfizer (1)',
       'Dosis entregadas Moderna (1)', 'Dosis entregadas AstraZeneca (1)',
       'Total Dosis entregadas (1)', 'Dosis administradas (2)',
       '% sobre entregadas', 'Nº Personas con al menos 1 dosis',
       'Nº Personas vacunadas(pauta completada)',
       'Fecha de la última vacuna registrada (2)'],
      dtype='object')

In [9]:
datos_hoy.columns=['ccaa',
                   'Dosis entregadas Pfizer',
                   'Dosis entregadas Moderna',
                   'Dosis entregadas AstraZeneca',
                   'Dosis entregadas',
                   'Dosis administradas',
                   '% sobre entregadas',
                   'Total 1 vacuna',
                   'Total pauta completada',
                   'Fecha de la ultima vacuna registrada']

In [10]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [11]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [12]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [13]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,1468440,219200,564800,2252440,1970183,0.874688,1419209,550974,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,291645,45800,88700,426145,331419,0.777714,228547,102872,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,280535,42200,68000,390735,317019,0.811340,216006,101013,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,158310,24300,81400,264010,233042,0.882701,175460,57582,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,319380,47800,145200,512380,418843,0.817446,302770,116073,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,132945,19900,40500,193345,147960,0.765264,101801,46159,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,647895,100800,166900,915595,730342,0.797669,492310,238032,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,412875,63800,142400,619075,499581,0.806980,344341,155240,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,1413980,221100,519100,2154180,1843205,0.855641,1381065,462140,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,836740,131600,352300,1320640,1125277,0.852069,835363,289914,13/4/21,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [14]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas','Total 1 vacuna',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [15]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [16]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla',
       'Fuerzas Armadas', 'Totales'], dtype=object)

In [17]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,14/4/21,Andalucía,1468440,219200,564800,2252440,1970183,0.874688,1419209,550974,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,14/4/21,Aragón,291645,45800,88700,426145,331419,0.777714,228547,102872,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,14/4/21,Asturias,280535,42200,68000,390735,317019,0.811340,216006,101013,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,14/4/21,Baleares,158310,24300,81400,264010,233042,0.882701,175460,57582,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,14/4/21,Canarias,319380,47800,145200,512380,418843,0.817446,302770,116073,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,14/4/21,Cantabria,132945,19900,40500,193345,147960,0.765264,101801,46159,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,14/4/21,Castilla y Leon,647895,100800,166900,915595,730342,0.797669,492310,238032,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,14/4/21,Castilla La Mancha,412875,63800,142400,619075,499581,0.806980,344341,155240,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,14/4/21,Cataluña,1413980,221100,519100,2154180,1843205,0.855641,1381065,462140,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,14/4/21,C. Valenciana,836740,131600,352300,1320640,1125277,0.852069,835363,289914,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [18]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [19]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [20]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,13/4/21,País Vasco,454950.0,70900.0,148200.0,674050,547803,0.812704,404891.0,142912.0,NaN,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1411,13/4/21,Ceuta,10170.0,1600.0,5800.0,17570,16211,0.922652,12566.0,3645.0,NaN,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1412,13/4/21,Melilla,10170.0,1200.0,5800.0,17170,14206,0.827373,10492.0,3714.0,NaN,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1413,13/4/21,Fuerzas Armadas,19500.0,0.0,66100.0,85600,68826,0.804042,63508.0,5318.0,NaN,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [21]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=False)

In [22]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total 1 vacuna', 'Total pauta completada',
       'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [23]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas','Total 1 vacuna',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [24]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,14/4/21,País Vasco,454950.0,70900.0,148200.0,674050,564370,0.837282,412035.0,152335.0,NaN,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
17,14/4/21,Ceuta,10170.0,1600.0,5800.0,17570,16764,0.954126,13058.0,3706.0,NaN,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
18,14/4/21,Melilla,10170.0,1200.0,5800.0,17170,14706,0.856494,10849.0,3857.0,NaN,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
19,14/4/21,Fuerzas Armadas,19500.0,0.0,66100.0,85600,69125,0.807535,63793.0,5332.0,NaN,13/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [25]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales',
       'Fuerzas Armadas'], dtype=object)

y volvemos a subirlo..

In [26]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

# Institucionalizados

In [27]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_institucionalizados_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Institucionalizados'

In [28]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [29]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210414.ods


In [30]:
datos_row

,Unnamed: 0,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA
0,Andalucía,41651,41160,39186,0.988212,0.940818
1,Aragón,18400,18289,17411,0.993967,0.946250
2,Asturias,11948,11755,11168,0.983847,0.934717
3,Baleares,5496,5156,4923,0.938137,0.895742
4,Canarias,10669,10669,9615,1.000000,0.901416
5,Cantabria,7470,7112,6550,0.952075,0.876841
6,Castilla y Leon,46000,45984,44327,0.999652,0.963630
7,Castilla - La Mancha,25566,24645,23182,0.963976,0.906751
8,Cataluña,75649,75321,63820,0.995664,0.843633
9,C. Valenciana,29605,29605,27396,1.000000,0.925384


In [31]:
datos_hoy = datos_row.copy()

In [32]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [33]:
datos_hoy.columns = ['ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA', 'date_pub',
       'source_name', 'source']

In [34]:
datos_hoy

,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,date_pub,source_name,source
0,Andalucía,41651,41160,39186,0.988212,0.940818,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,18400,18289,17411,0.993967,0.946250,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,11948,11755,11168,0.983847,0.934717,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,5496,5156,4923,0.938137,0.895742,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,10669,10669,9615,1.000000,0.901416,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,7470,7112,6550,0.952075,0.876841,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,46000,45984,44327,0.999652,0.963630,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla - La Mancha,25566,24645,23182,0.963976,0.906751,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,75649,75321,63820,0.995664,0.843633,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,29605,29605,27396,1.000000,0.925384,14/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [35]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA',
       'source_name', 'source']]

In [36]:
datos_hoy

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,14/4/21,Andalucía,41651,41160,39186,0.988212,0.940818,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,14/4/21,Aragón,18400,18289,17411,0.993967,0.946250,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,14/4/21,Asturias,11948,11755,11168,0.983847,0.934717,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,14/4/21,Baleares,5496,5156,4923,0.938137,0.895742,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,14/4/21,Canarias,10669,10669,9615,1.000000,0.901416,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,14/4/21,Cantabria,7470,7112,6550,0.952075,0.876841,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,14/4/21,Castilla y Leon,46000,45984,44327,0.999652,0.963630,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,14/4/21,Castilla - La Mancha,25566,24645,23182,0.963976,0.906751,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,14/4/21,Cataluña,75649,75321,63820,0.995664,0.843633,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,14/4/21,C. Valenciana,29605,29605,27396,1.000000,0.925384,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [37]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_20210414'

In [38]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

..y ahora lo merge a los acumulados::

In [39]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [40]:
datos_acumulados

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,31/3/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
195,13/4/21,Navarra,8343,8057,7864,0.965720,0.942587,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
196,13/4/21,País Vasco,23000,22924,20333,0.996696,0.884043,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
197,13/4/21,Ceuta,210,195,189,0.928571,0.900000,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
198,13/4/21,Melilla,315,305,292,0.968254,0.926984,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [41]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [42]:
datos_acumulados

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,31/3/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
215,14/4/21,Navarra,8343,8077,7915,0.968117,0.948700,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
216,14/4/21,País Vasco,23500,23293,20499,0.991191,0.872298,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
217,14/4/21,Ceuta,198,195,189,0.984848,0.954545,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
218,14/4/21,Melilla,325,317,295,0.975385,0.907692,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


y volvemos a subirlo:

In [43]:
nombre_fichero_acumulado

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_'

In [44]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [45]:
nombre_fichero_acumulado_long = '/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_long'
pd.melt(datos_acumulados,id_vars=['date_pub','ccaa','source_name','source'],value_vars=[
    'Denominador Poblacional CCAA',
    'Persona institucionalizada con al menos 1 dosis (2)',
    'Persona institucionalizada  Pauta Completa (2) ',
    '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
    '% Pautas Completas sobre Denominador Poblacional CCAA']
                                      ).to_csv(nombre_fichero_acumulado_long+'.csv',index=False)

# Etarios 1 dosis

In [46]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_1dosis_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=3,day=31)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_al_menos_1_dosis'

In [47]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [48]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210414.ods


In [49]:
datos_hoy = datos_row.copy()

In [50]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [51]:
datos_hoy.columns = ['ccaa', 'Personas con al menos 1 dosis >=80 años',
       'Población INE>=80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [52]:
datos_hoy = datos_hoy [['date_pub','ccaa', 'Personas con al menos 1 dosis >=80 años',
       'Población INE>=80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 
       'source_name', 'source']]

In [53]:
datos_hoy

,date_pub,ccaa,Personas con al menos 1 dosis >=80 años,Población INE>=80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,14/4/21,Andalucía,415210,413516,1.004097,245253,650381,0.377091,256419,916147,...,625846,0.045593,247,181921,0.001358,1419209,7062213,0.200958,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,14/4/21,Aragón,96635,98807,0.978018,15413,118955,0.129570,34368,156367,...,89180,0.064465,30,25106,0.001195,228547,1132764,0.201760,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,14/4/21,Asturias,85312,87826,0.971375,37732,109249,0.345376,37912,149652,...,54179,0.056369,34,15563,0.002185,216006,901209,0.239685,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,14/4/21,Baleares,47082,49542,0.950345,18995,80516,0.235916,42756,117231,...,85839,0.042999,33,23300,0.001416,175460,986279,0.177901,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,14/4/21,Canarias,79717,90236,0.883428,42407,154531,0.274424,79929,235092,...,163899,0.035485,39,44352,0.000879,302770,1871033,0.161820,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,14/4/21,Cantabria,38555,41042,0.939404,15967,53066,0.300889,12093,77345,...,34935,0.044712,5,10791,0.000463,101801,501384,0.203040,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,14/4/21,Castilla y Leon,218983,223657,0.979102,48495,245410,0.197608,79163,318575,...,144422,0.051211,93,40684,0.002286,492310,2092873,0.235232,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,14/4/21,Castilla - La Mancha,130188,135194,0.962972,23131,159101,0.145386,65155,218444,...,149518,0.039701,93,42364,0.002195,344341,1719964,0.200202,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,14/4/21,Cataluña,426748,447196,0.954275,150499,635292,0.236897,346401,837584,...,556316,0.063295,311,159864,0.001945,1381065,6531658,0.211442,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,14/4/21,C. Valenciana,282167,276531,1.020381,110681,437862,0.252776,135803,580728,...,352834,0.049332,114,103049,0.001106,835363,4269305,0.195667,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [54]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_1dosis_20210414'

In [55]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora merge con los acumulados:

In [56]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [57]:
#datos_acumulados.columns = datos_hoy.columns

In [58]:
datos_acumulados

,date_pub,ccaa,Personas con al menos 1 dosis >=80 años,Población INE>=80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,375382,413516,0.9077810773948287,21914,650381,0.03369409622974841,60757,916147,...,625846,0.03992036379556632,144,181921,0.0007915523771307326,933984,7062213,0.13225089642580873,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,71256,98807,0.7211634803202203,3553,118955,0.029868437644487413,11417,156367,...,89180,0.06159452792105853,28,25106,0.0011152712499004221,161061,1132764,0.14218407364640825,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,67037,87826,0.763293329993396,4258,109249,0.03897518512755266,15803,149652,...,54179,0.05197585780468447,30,15563,0.0019276489108783653,138822,901209,0.15403973994933473,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,33141,49542,0.6689475596463607,2346,80516,0.029137065924785138,15295,117231,...,85839,0.040972052330525754,31,23300,0.0013304721030042918,115375,986279,0.11698008372884346,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,54220,90236,0.600868832838335,15913,154531,0.10297610188247018,33435,235092,...,163899,0.034124674342125336,27,44352,0.0006087662337662338,199517,1871033,0.1066346772077243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,13/4/21,País Vasco,147289,157950,0.9325039569484014,29339,213081,0.13768942327096267,77171,280119,...,140323,0.08188251391432623,7,41824,0.0001673680183626626,404891,1898123,0.21331125538229082,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
206,13/4/21,Ceuta,2209,2605,0.8479846449136277,2405,4318,0.5569708198239925,2332,7857,...,7581,0.03719825880490701,0,2175,0.0,12566,66158,0.18993923637353002,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
207,13/4/21,Melilla,1655,2349,0.7045551298424861,2195,3748,0.5856456776947705,1687,7827,...,8536,0.02261012183692596,6,2324,0.0025817555938037868,10492,66178,0.15854211369337243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
208,13/4/21,Fuerzas Armadas,27,-,-,8,-,-,2296,-,...,-,-,0,-,-,63508,-,-,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [59]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [60]:
datos_acumulados

,date_pub,ccaa,Personas con al menos 1 dosis >=80 años,Población INE>=80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,375382,413516,0.9077810773948287,21914,650381,0.03369409622974841,60757,916147,...,625846,0.03992036379556632,144,181921,0.0007915523771307326,933984,7062213,0.13225089642580873,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,71256,98807,0.7211634803202203,3553,118955,0.029868437644487413,11417,156367,...,89180,0.06159452792105853,28,25106,0.0011152712499004221,161061,1132764,0.14218407364640825,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,67037,87826,0.763293329993396,4258,109249,0.03897518512755266,15803,149652,...,54179,0.05197585780468447,30,15563,0.0019276489108783653,138822,901209,0.15403973994933473,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,33141,49542,0.6689475596463607,2346,80516,0.029137065924785138,15295,117231,...,85839,0.040972052330525754,31,23300,0.0013304721030042918,115375,986279,0.11698008372884346,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,54220,90236,0.600868832838335,15913,154531,0.10297610188247018,33435,235092,...,163899,0.034124674342125336,27,44352,0.0006087662337662338,199517,1871033,0.1066346772077243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,14/4/21,País Vasco,148393,157950,0.939494,35074,213081,0.164604,77238,280119,...,140323,0.082146,7,41824,0.000167,412035,1898123,0.217075,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
227,14/4/21,Ceuta,2222,2605,0.852975,2823,4318,0.653775,2363,7857,...,7581,0.037594,0,2175,0.000000,13058,66158,0.197376,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
228,14/4/21,Melilla,1679,2349,0.714772,2314,3748,0.617396,1893,7827,...,8536,0.022610,6,2324,0.002582,10849,66178,0.163937,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
229,14/4/21,Fuerzas Armadas,26,-,-,9,-,-,2508,-,...,-,-,0,-,-,63793,-,-,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [61]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [62]:
nombre_fichero_acumulado_long = '/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_1dosis_long'
pd.melt(datos_acumulados,id_vars=['date_pub','ccaa','source_name','source'],value_vars=[
    'Personas con al menos 1 dosis >=80 años',
       'Población INE>=80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE']
                                      ).to_csv(nombre_fichero_acumulado_long+'.csv',index=False)

# Etarios con pauta completa

In [63]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_pauta_completa_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_pauta_completa'

In [64]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [65]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210414.ods


In [66]:
datos_hoy = datos_row.copy()

In [67]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [68]:
datos_hoy.columns = ['ccaa', 'Personas pauta completa >=80 años',
       'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [69]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Personas pauta completa >=80 años',
       'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE',
       'source_name', 'source']]

In [70]:
datos_hoy

,date_pub,ccaa,Personas pauta completa >=80 años,Población INE>=80 años,%,Personas pauta completa 70-79 años,Población INE70-79 años,%.1,Personas pauta completa 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas pauta completa,Total Población INE Población a Vacunar (1),% pauta completa sobre Población a Vacunar INE,source_name,source
0,14/4/21,Andalucía,275864,413516,0.667118,14981,650381,0.023034,42932,916147,...,625846,0.019407,102,181921,0.000561,550974,7062213,0.078017,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,14/4/21,Aragón,52638,98807,0.532736,3315,118955,0.027868,9550,156367,...,89180,0.016338,25,25106,0.000996,102872,1132764,0.090815,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,14/4/21,Asturias,49008,87826,0.558012,3839,109249,0.035140,10334,149652,...,54179,0.044298,27,15563,0.001735,101013,901209,0.112086,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,14/4/21,Baleares,23743,49542,0.479250,2231,80516,0.027709,5030,117231,...,85839,0.014690,30,23300,0.001288,57582,986279,0.058383,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,14/4/21,Canarias,38034,90236,0.421495,13946,154531,0.090247,9569,235092,...,163899,0.014716,21,44352,0.000473,116073,1871033,0.062037,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,14/4/21,Cantabria,23855,41042,0.581234,1704,53066,0.032111,3768,77345,...,34935,0.017575,3,10791,0.000278,46159,501384,0.092063,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,14/4/21,Castilla y Leon,132837,223657,0.593932,9873,245410,0.040231,20251,318575,...,144422,0.018377,86,40684,0.002114,238032,2092873,0.113735,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,14/4/21,Castilla - La Mancha,74245,135194,0.549174,5809,159101,0.036511,13857,218444,...,149518,0.023957,85,42364,0.002006,155240,1719964,0.090258,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,14/4/21,Cataluña,171122,447196,0.382655,28724,635292,0.045214,49678,837584,...,556316,0.029701,215,159864,0.001345,462140,6531658,0.070754,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,14/4/21,C. Valenciana,147457,276531,0.533239,10563,437862,0.024124,23901,580728,...,352834,0.014894,101,103049,0.000980,289914,4269305,0.067907,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [71]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_20210414'

In [72]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora los merge con los acumulados

In [73]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [74]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [75]:
#datos_acumulados.columns = datos_hoy.columns

In [76]:
datos_acumulados

,date_pub,ccaa,Personas pauta completa >=80 años,Población INE>=80 años,%,Personas pauta completa 70-79 años,Población INE70-79 años,%.1,Personas pauta completa 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas pauta completa,Total Población INE Población a Vacunar (1),% pauta completa sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,240454,413516,0.5814865688389325,12788,650381,0.019662321008762557,41807,916147,...,625846,0.018677118652192393,83,181921,0.0004562419951517417,507430,7062213,0.0718514154132706,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,40245,98807,0.40730919874097987,3048,118955,0.025623134798873525,9273,156367,...,89180,0.015934065934065933,25,25106,0.0009957779016968055,89278,1132764,0.07881429847699963,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,37353,87826,0.4253068567394621,3490,109249,0.031945372497688765,9709,149652,...,54179,0.04263644585540523,24,15563,0.0015421191287026924,87093,901209,0.09664018002483331,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,12403,49542,0.2503532356384482,2088,80516,0.025932733866560684,4856,117231,...,85839,0.014317501368841669,30,23300,0.0012875536480686696,45509,986279,0.04614211597326923,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,31183,90236,0.34557161221685356,11216,154531,0.07258090609651138,9016,235092,...,163899,0.014331997144582944,19,44352,0.0004283910533910534,104712,1871033,0.055964806606831626,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,14/4/21,País Vasco,75547,157950,0.478297,3730,213081,0.017505,12866,280119,...,140323,0.022234,6,41824,0.000143,152335,1898123,0.080256,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
227,14/4/21,Ceuta,1632,2605,0.626488,383,4318,0.088698,315,7857,...,7581,0.005804,0,2175,0.000000,3706,66158,0.056017,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
228,14/4/21,Melilla,1381,2349,0.587910,706,3748,0.188367,342,7827,...,8536,0.006795,6,2324,0.002582,3857,66178,0.058282,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
229,14/4/21,Fuerzas Armadas,1,-,-,4,-,-,471,-,...,-,-,0,-,-,5332,-,-,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [77]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Personas pauta completa >=80 años',
       'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE', 'source_name',
       'source'],
      dtype='object')

In [78]:
nombre_fichero_acumulado

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_'

In [79]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [80]:
nombre_fichero_acumulado_long = '/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_long'
pd.melt(datos_acumulados,id_vars=['date_pub','ccaa','source_name','source'],value_vars=[
    'Personas pauta completa >=80 años',
    'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
    'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
    'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
    'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
    'Población INE25-49 años', '%.4', '18-24 años',
    'Población INE18-24 años', '%.5', '16-17 años',
    'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
    'Total Población INE Población a Vacunar (1)',
    '% pauta completa sobre Población a Vacunar INE']
                                      ).to_csv(nombre_fichero_acumulado_long+'.csv',index=False)